In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
CLIENT_ID = "0672c46c9efb4ccfaaa7133ef0476716"
CLIENT_SECRET = "8102d349542f4dfeacf7c8358b0643f3"
credentials = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=credentials)
print(sp)

In [3]:
def get_playlist_tracks(sp, playlist_id):
    tracks = sp.playlist_tracks(playlist_id, fields="items.track.id,total")
    track_ids = [item["track"]["id"] for item in tracks["items"]]
    return track_ids


def get_audio_features(sp, track_ids):
    audio_features = sp.audio_features(track_ids)
    return audio_features


def audio_features_to_df(audio_features):
    # Define the relevant numerical features to include in the dataframe
    relevant_features = [
        "danceability",
        "energy",
        "key",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "duration_ms",
    ]

    # Convert the list of dictionaries to a dataframe
    df = pd.DataFrame(audio_features)
    df = df[relevant_features]

    return df


def calculate_similarity(playlist_audio_features):
    playlist_dfs = []
    for playlist_features in playlist_audio_features:
        playlist_dfs.append(audio_features_to_df(playlist_features))

    combined_df = pd.concat(playlist_dfs)

    similarity_matrix = cosine_similarity(combined_df)
    overall_similarity = np.mean(similarity_matrix)

    # Calculate the mean similarity for each feature
    feature_mean_similarity = np.mean(similarity_matrix, axis=0)

    return overall_similarity, feature_mean_similarity

In [4]:
playlist_ids = [
    "7ywZVoADC6voXoQH3dmwev",
    "4hbN2UXfG3ToOw3IWoGWLW",
    "58lFQMNXNmgnmXRH3u7xS8",
    "3v7NNKF03kDnjVgZElbzD5",
    "3urbJyAzlYtxPD1qYD12Zf",
    "10Vz2pd5IEGGIGFldUQXPS",
    "5k5W4hNyxHGcol9Rj61s5b",
    "3cu63OCX6DCKNrDjwpNK8U",
    "7LsTkj73rKbgHbGpwHLR0R",
    "2Q02CP199hpLvNr2VOF8nF",
    "3VEjp8e4rlHwPmeIlrP62Q",
]

playlist_audio_features = []
for playlist_id in playlist_ids:
    track_ids = get_playlist_tracks(sp, playlist_id)
    playlist_audio_features.append(get_audio_features(sp, track_ids))

overall_similarity, feature_mean_similarity = calculate_similarity(
    playlist_audio_features
)
print(f"The overall similarity between the playlists is: {overall_similarity}")

# Print the mean similarity for each song feature
feature_names = [
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
]
for feature, mean_similarity in zip(feature_names, feature_mean_similarity):
    print(f"Mean similarity for feature '{feature}': {mean_similarity}")

The overall similarity between the playlists is: 0.9999999607151101
Mean similarity for feature 'danceability': 0.9999999759289032
Mean similarity for feature 'energy': 0.9999999658978739
Mean similarity for feature 'key': 0.9999999802590567
Mean similarity for feature 'loudness': 0.9999999724248905
Mean similarity for feature 'mode': 0.9999999672973872
Mean similarity for feature 'speechiness': 0.9999999753601277
Mean similarity for feature 'acousticness': 0.9999999801501638
Mean similarity for feature 'instrumentalness': 0.9999999660185404
Mean similarity for feature 'liveness': 0.9999999798344735
Mean similarity for feature 'valence': 0.999999976088365
Mean similarity for feature 'tempo': 0.999999972431342
Mean similarity for feature 'duration_ms': 0.9999999784319331
